In [1]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os
#import pyreadr

In [13]:
import pyreadr
ligands_readin = pyreadr.read_r("ligands_human.rda")
ligands = ligands_readin["ligands_human"]
ligands.to_csv("ligands_human.csv", index=False)

ligands_readin_m = pyreadr.read_r("converted_ligands.rda")
print (ligands_readin_m)
ligands_m = ligands_readin_m["converted_ligands"]
ligands_m.to_csv("converted_ligands.csv", index=False)

rtf_readin = pyreadr.read_r("RTF_DB_2.rda") 
print(rtf_readin)
rtf_db = rtf_readin["RTF_DB_2"]
rtf_db.to_csv("rtf_db.csv", index=False)

rtf_m_readin = pyreadr.read_r("RTF_DB_mouse.rda") 
print (rtf_m_readin)
rtf_db_m = rtf_m_readin["RTF_DB_mouse"]
rtf_db_m.to_csv("rtf_db_mouse.csv", index=False)

OrderedDict({'converted_ligands':     converted_ligands
0              Col6a6
1              Col6a5
2               Ccl24
3              Lgals8
4                Cort
..                ...
693             Insl5
694              Wif1
695              Wnt6
696             Lamc3
697              Icos

[698 rows x 1 columns]})
OrderedDict({'RTF_DB_2':       receptor      tf
0         ABL1    AIRE
1         ABL1   ARNTL
2         ABL1    ATF2
3         ABL1   CEBPB
4         ABL1   CREB1
...        ...     ...
16130    ZAP70    TAL1
16131    ZAP70   TERF1
16132    ZAP70  TWIST1
16133    ZAP70    YBX1
16134    ZAP70  ZNF322

[16135 rows x 2 columns]})
OrderedDict({'RTF_DB_mouse':       receptor      tf
0         Abl1    Aire
1         Abl1   Arntl
2         Abl1    Atf2
3         Abl1   Cebpb
4         Abl1   Creb1
...        ...     ...
14254    Zap70  Stat5a
14255    Zap70  Stat5b
14256    Zap70    Tal1
14257    Zap70   Terf1
14258    Zap70  Twist1

[14259 rows x 2 columns]})


In [ ]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None):

  ligands = pd.read_csv("ligands_human.csv")
  R2TF = pd.read_csv("rtf_db.csv")

  sorted_regulon = regulon[["tf", "target"]]
  #sorted_regulon.rename(columns={"target" : "targets"})
  sorted_regulon = sorted_regulon.set_index("tf")

  R2TF = R2TF.set_index("tf")
  output_df = create_empty_CTR_dataframe()
  tf_l = create_empty_CTR_dataframe()
  r_tf = create_empty_CTR_dataframe()

  for row in range(len((tf_activities))):

    #if (tf_activities["z_score"].iloc[row] > 0):
    tf = str(tf_activities["gene"].iloc[row])
    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]

    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]

    else:
        receptors = []
    tf_ligands = np.intersect1d(targets, ligands)

    if len(tf_ligands) > 0:
      for ligand in tf_ligands:
          expressed = False
          if ligand in gene_expression.index:
            ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
            if (ex_value != 0):
              expressed = True
          
          df_list_l = []
            
          if (expressed == True):
            df_list_l = add_entry_to_CTR_dataframe(source = tf_activities.iloc[row, 2],
                                                    target = tf_activities.iloc[row, 2],
                                                    gene_A =tf_activities.iloc[row, 0],
                                                    gene_B = ligand,
                                                    type_gene_A = "Transcription_Factor",
                                                    type_gene_B= "Ligand",
                                                    MeanLR= tf_activities.iloc[row, 3],
                                                    row= row)
              
            tf_l = pd.concat([tf_l, df_list_l], ignore_index=True)
            
    
      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = [] 
          df_list_r = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
                                              tf_activities.iloc[row, 2],
                                              receptor,
                                              tf_activities.iloc[row, 0],
                                              "Receptor",
                                              "Transcription Factor",
                                              tf_activities.iloc[row, 3],
                                              row)
              
          print(df_list_r)
            
          r_tf = pd.concat([r_tf, df_list_r], ignore_index=True)

      #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
      #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)

  #    r_tf["gene_A"] <- gsub("_", "+", r_tf$gene_A, fixed = TRUE)
   #   r_tf["gene_B"] <- gsub("_", "+", r_tf$gene_B, fixed = TRUE)
   #   tf_l["gene_A"] <- gsub("_", "+", tf_l$gene_A, fixed = TRUE)
   #   tf_l["gene_B"] <- gsub("_", "+", tf_l$gene_B, fixed = TRUE)

  output_df = pd.concat([output_df, r_tf], ignore_index=True)
  output_df = pd.concat([output_df, tf_l], ignore_index=True)
  #output_df.to_csv("tf_l_r.csv", index=0)


  return output_df

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    z_score = []
    for gene in anndataobject_markers["gene"]:
        if gene in filtered_summarized_tf_scores_df.index:
            z_score.append(filtered_summarized_tf_scores_df.loc[gene, anndataobject_markers["cluster"]])
    return z_score


In [ ]:
#comparison python wilcoxon and r wilcoxon 

py_result_wilcx = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

py_result_wilcx = py_result_wilcx[py_result_wilcx["pvals_adj"] < 0.05]
r_result_wilcx = r_result_wilcx[r_result_wilcx["p_val_adj"] < 0.05]

py_result_wilcx = py_result_wilcx.rename(columns={"names": "gene", "group": "cluster"})

df_output_py = pd.DataFrame()

for i in range(len(py_result_wilcx)):
    a = py_result_wilcx["gene"].iloc[i]
    for j in range(len(r_result_wilcx)):
        b = r_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result_wilcx.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result_wilcx)):
    a = r_result_wilcx["gene"].iloc[i]
    for j in range(len(py_result_wilcx)):
        b = py_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result_wilcx.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_wilcoxon_r_same_genes_control_filtered.csv")

#comparison python t test and r wilcoxon

py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

#df_output = pd.concat([df_output_py, df_output_r], keys= ["gene"])
df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_t_test_r_same_genes_control_filtered.csv")

# py wilcox vs py t test
py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")


py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
df_output.to_csv("tf_comparison_r_py/py_t_test_py_wilcox_same_genes_control_filtered.csv")

In [ ]:

    
    #  df = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 0],
    #                                         ligand,
    #                                         'Transcription Factor',
    #                                         'Ligand',
     #                                        tf_activities.iloc[row, 3])
      
 #     

In [ ]:
gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
obs = sub_object[:,gene_ids].X.toarray()
obs = np.expm1(obs)
sub_object = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs["new_annotation"])
sub_object = sub_object.groupby(level=0, observed=False).mean()
#sub_object.T.to_csv(outpath + name + "average_gene_expression_by_cluster_exp.csv")

sub_object.T


In [25]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
      #  anndataobject.rename_categories("protocol", ["PMFMF2", "control"])

tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs

tf_activities_sub = tf_activities[anndataobject.obs.protocol == "PMF,MF2"]
#print(tf_activities_sub.var.index.values)
#print(tf_activities_sub.var_names)

tf_act_2 = tf_activities_sub
tf_act_2.obs_names = tf_act_2.obs["new_annotation"]
tf_df = tf_act_2.to_df()
tf_df.groupby("new_annotation").mean().T

d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\anndata\_core\anndata.py:841: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    ['Neural', 'MSC', 'Megakaryocyte', 'Neural', 'Neural']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "obs")
C:\Users\Larissa\AppData\Local\Temp\ipykernel_21852\3832065385.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tf_df.groupby("new_annotation").mean().T


new_annotation,Neural,MSC,Fibroblast,Megakaryocyte,Myeloid
ADNP,-0.241832,-0.638793,-0.870116,-0.956318,-1.173983
AHR,-0.969373,1.512223,-0.779942,-1.687008,-1.425144
AR,0.431653,1.157381,-0.045079,-0.124143,-0.186571
ARID2,0.924855,1.196451,0.000928,0.239218,-0.520630
ARID3A,0.130700,0.851371,-0.793377,-0.267953,-1.128399
...,...,...,...,...,...
ZNF750,-1.572268,-0.265979,-1.877062,-1.669147,-1.741956
ZNF766,-0.038935,-0.655792,-0.877722,-0.825913,-0.900506
ZNF83,1.349940,0.266376,0.431817,1.134436,0.747866
ZNF92,-1.194463,-0.231719,-1.322473,-1.795834,-1.618380


In [ ]:
tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs
tf_activities = tf_activities[tf_activities.obs.protocol == "control"]
tf_activities.obs

In [ ]:
#tag mapping t over
    tag_mapping_t_over= anndataobject_markers_t_over[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]

    tag_mapping_t_over = tag_mapping_t_over[(tag_mapping_t_over["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_t_over["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_t_over["logfoldchanges"] < -float(logfc)))]

     # Pivot the DataFrame to create a wide format similar to `dcast` in R
    tag_mapping_t_over = tag_mapping_t_over.pivot(index="gene", columns="cluster", values="tag")
    tag_mapping_t_over.fillna("ns", inplace=True)

In [ ]:
# Summarize viper_scores_clusters by TF and cell type: compute mean and std (standard deviation)
    #summarized_viper_scores = (viper_scores_clusters
       # .groupby(['tf', 'cell_type'])  # Group by transcription factor and cell type
     #   .agg(avg=('activity', 'mean'), std=('activity', 'std'))  # Calculate mean and standard deviation
      #  .reset_index())

# Spread the summarized data to wide format: remove 'std', and spread 'tf' on columns, 'avg' as values
   # summarized_viper_scores_df = (summarized_viper_scores
   #     .drop(columns=['std'])  # Remove the std column
   #     .pivot(index='cell_type', columns='tf', values='avg')  # Spread the TFs as columns, average activity as values
    #    .reset_index(drop=True))  # Reset the index to make it a regular DataFrame

# Transpose the dataframe (same as R's t())
   # summarized_viper_scores_df = summarized_viper_scores_df.T

# Write the summarized TF scores to CSV

In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")


tf_activities = ad.read_csv("decoupler_results.csv")
anndataobject.obsm['ulm_estimate'] = tf_activities
print(anndataobject.obsm["ulm_estimate"].X)
for name in anndataobject.obs.protocol.unique():
##    subobject = anndataobject[anndataobject.obs.protocol == name]
#print(type(subobject)) 
#print(type(anndataobject))  
#print(subobject.X)

